# Series potential

In [ ]:
from setproctitle import setproctitle
setproctitle("Potential")

In [ ]:
import os
import glob 
import time
import numpy as np 
from cmspinn.potential_field import potential_cube

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

In [ ]:
b_bottom_paths = os.path.expanduser('~/workspace/_data/NOAA12673/b_bottom')

In [ ]:
b_potential_paths = os.path.expanduser('~/workspace/_data/NOAA12673/b_potential')
os.makedirs(b_potential_paths, exist_ok=True)

In [ ]:
Nz = 160

In [ ]:
start = time.time()

for b_bottom_path in sorted(glob.glob(os.path.join(b_bottom_paths, '*.npy'))):

    b_bottom_date = os.path.basename(b_bottom_path)[9:-4]
    
    with open(b_bottom_path, 'rb') as f:
        b_bottom = np.load(f)

    bz_bottom = b_bottom[:, :, 2]
    potential_maker = potential_cube(bz_bottom, Nz)

    vtk_path = os.path.join(b_potential_paths, f'b_potential_{b_bottom_date}.vtk')

    if os.path.exists(vtk_path):
        print(f"Already exists: {vtk_path}")
        continue

    potential_maker.cal_and_save_potential_field(vtk_path, batch_size=10000)
    
    print(vtk_path)

runtime = time.time() - start
print(f'Runtime: {runtime:.2f} sec')